In [ ]:
# 이건 일단 보류(일단 실행 되지만 확인)(2순위 점검)
import sqlite3
import numpy as np
import mediapipe as mp

# SQLite 데이터베이스 초기화
def init_db():
    conn = sqlite3.connect('face_landmarks.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS faces (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        landmarks TEXT)''')
    conn.commit()
    conn.close()

# 얼굴 랜드마크 DB에 저장
def save_landmarks_to_db(name, landmarks):
    conn = sqlite3.connect('face_landmarks.db')
    cursor = conn.cursor()
    
    # 랜드마크를 문자열로 변환 (튜플 목록을 문자열로 변환)
    landmarks_str = str(landmarks)
    cursor.execute("INSERT INTO faces (name, landmarks) VALUES (?, ?)", (name, landmarks_str))
    conn.commit()
    conn.close()

# DB에서 랜드마크 가져오기
def get_landmarks_from_db(name):
    conn = sqlite3.connect('face_landmarks.db')
    cursor = conn.cursor()
    cursor.execute("SELECT landmarks FROM faces WHERE name=?", (name,))
    result = cursor.fetchone()
    conn.close()
    
    if result:
        landmarks_str = result[0]
        landmarks = eval(landmarks_str)  # 문자열을 리스트로 변환
        return landmarks
    return None


In [ ]:
# 얼굴 인식 및 랜드마크 추출 후 저장
import cv2
import mediapipe as mp

# MediaPipe 설정
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 추출
def extract_face_landmarks(frame):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.2, min_tracking_confidence=0.2) as face_mesh:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb_frame)
        
        if result.multi_face_landmarks:
            landmarks = []
            for landmarks_list in result.multi_face_landmarks:
                for landmark in landmarks_list.landmark:
                    x = int(landmark.x * frame.shape[1])
                    y = int(landmark.y * frame.shape[0])
                    landmarks.append((x, y))
            return landmarks
    return None

# 얼굴을 인식하고 랜드마크 저장
def capture_and_save_landmarks():
    name = input("얼굴 인증을 위한 이름을 입력하세요: ")
    
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        landmarks = extract_face_landmarks(frame)
        if landmarks:
            # 랜드마크 추출 후 DB에 저장
            save_landmarks_to_db(name, landmarks)
            print("얼굴 랜드마크가 저장되었습니다.")
            break

        # 얼굴 랜드마크를 화면에 표시
        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# DB 초기화
init_db()

# 랜드마크 저장 테스트
capture_and_save_landmarks() # 경로 지정


test용 코드입니다

In [4]:
import cv2
import mediapipe as mp

# MediaPipe 설정
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크 추출
def extract_face_landmarks_from_image(image_path):
    # 사진 읽기
    img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 얼굴 랜드마크 추출
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.2, min_tracking_confidence=0.2) as face_mesh:
        result = face_mesh.process(rgb_img)
        
        if result.multi_face_landmarks:
            landmarks = []
            for landmarks_list in result.multi_face_landmarks:
                for landmark in landmarks_list.landmark:
                    x = int(landmark.x * img.shape[1])
                    y = int(landmark.y * img.shape[0])
                    landmarks.append((x, y))
            return landmarks
    return None

# 사진 경로에 맞게 변경
image_path = "C:/Users/user/Desktop/python/enter/user.png"  # 자신의 얼굴 사진 경로 입력

# 얼굴 랜드마크 추출
landmarks_from_image = extract_face_landmarks_from_image(image_path)

if landmarks_from_image:
    print(f"추출된 얼굴 랜드마크: {landmarks_from_image}")
else:
    print("얼굴을 인식할 수 없습니다.")


추출된 얼굴 랜드마크: [(671, 492), (668, 454), (668, 465), (656, 421), (667, 443), (665, 430), (662, 396), (589, 400), (659, 371), (657, 357), (652, 306), (672, 498), (672, 502), (672, 504), (673, 508), (673, 513), (674, 519), (675, 526), (675, 535), (668, 459), (659, 459), (545, 363), (619, 407), (609, 409), (599, 409), (586, 404), (627, 403), (603, 382), (613, 382), (593, 384), (587, 388), (579, 412), (638, 552), (585, 397), (547, 409), (566, 405), (617, 451), (659, 491), (662, 503), (647, 496), (640, 502), (654, 504), (647, 506), (629, 520), (659, 454), (657, 445), (569, 378), (633, 423), (636, 454), (634, 449), (588, 453), (656, 432), (592, 365), (578, 370), (553, 344), (637, 370), (623, 384), (621, 511), (569, 505), (644, 459), (652, 462), (632, 511), (637, 510), (571, 363), (636, 459), (611, 365), (608, 356), (593, 316), (562, 353), (601, 336), (562, 374), (554, 369), (660, 498), (651, 501), (644, 504), (646, 461), (635, 511), (639, 512), (638, 509), (649, 455), (649, 506), (656, 505), (6

In [5]:
def compare_landmarks(landmarks1, landmarks2):
    # 랜드마크 비교 (유클리드 거리 기준)
    landmarks1 = np.array(landmarks1)
    landmarks2 = np.array(landmarks2)
    
    # 두 랜드마크 간의 평균 유클리드 거리 계산
    distances = np.linalg.norm(landmarks1 - landmarks2, axis=1)
    similarity = np.mean(distances)
    return similarity

def authenticate_face(frame, name):
    # 실시간으로 얼굴 랜드마크 추출
    landmarks = extract_face_landmarks(frame)
    if landmarks:
        # DB에서 등록된 얼굴 랜드마크 가져오기
        saved_landmarks = get_landmarks_from_db(name)
        
        if saved_landmarks:
            # 랜드마크 비교
            similarity = compare_landmarks(saved_landmarks, landmarks)
            print(f"얼굴 유사도: {similarity}")

            # 유사도가 일정 threshold 이상이면 인증
            if similarity < 10:
                print("인증 성공!")
            else:
                print("비인증 얼굴입니다.")
        else:
            print("등록된 얼굴이 없습니다.")
    return landmarks

# 실시간 얼굴 인증
def capture_and_authenticate():
    name = input("인증을 위한 이름을 입력하세요: ")
    
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        
        landmarks = authenticate_face(frame, name)

        # 얼굴 랜드마크를 화면에 표시
        for (x, y) in landmarks:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

        # 인증 결과 표시
        cv2.imshow("Face Recognition Authentication", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 실시간 인증 시작
capture_and_authenticate()


NameError: name 'get_landmarks_from_db' is not defined